## Лабораторные занятия по численным методам

|                       |                             |
|:----------------------|:---------------------------:|
| Подготовили           | Смирнова Л. и Глуховский М. |
| Преподаватель         | Шабунина Зоя Александровна  |
| Лабораторная работа   |      № 1 Задание 12**       |
| Язык программирования |           Python            |

---

### Задание 12 (**).
**Назначение.** Численные исследования сходимости глобальных интерполяционных процессов для непрерывных на отрезке функций.

**Метод.** Для визуального исследования сходимости интерполяционных процессов разработать процедуру, которая выводит на экран компьютера два графика на заданном отрезке – график заданной функции
`f(x)` и график глобального интерполяционного многочлена Фейера,
построенного для этой функции на сетке чебышевских узлов. Входными параметрами этой процедуры являются концы отрезка интерполирования, количество чебышевских узлов на этом отрезке, непрерывная функция `f(x)`.

**Замечание.**
Вычислительные эксперименты должны быть проведены в том числе
для функций `1/(1 + 25x²)`, `|x|`.

**Указание.** См. `[1]` настоящего пособия.

---


In [1]:
# Подключение бибылиотек
from pydantic import BaseModel, PositiveInt, ValidationError, root_validator
import numpy as np
import scipy
import matplotlib.pyplot as plt
import sympy as sp
from sympy.abc import a, b, x, i, n
from sympy import cos, pi, abc

from pathlib import Path

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%matplotlib inline
%config IPCompleter.greedy=True

In [2]:
class InpData(BaseModel):
    a: float | int
    b: float | int
    number_nodes: PositiveInt
        
    @root_validator
    def check_segment_a_b(cls, values):
        a, b = values.get('a'), values.get('b')
        assert a <= b, f'invalid segment [{a}, {b}]'
        return values

try:
    inp_data = InpData.parse_file(Path('input.json'))
except ValidationError as e:
    print("ОШИБКА ВХОДНЫХ ДАННЫХ")
    print(e.json())
    raise e

### Функция

In [ ]:
# f_x = 1/ (1 + 25 * x**2)
f_x = x ** 7 - x**6 + 5*x**5 - 3*x**4 + x
f_x

### Чебышевские узлы

In [ ]:
def get_node(a: sp.core.symbol.Symbol | float | int = sp.symbols('a'), 
             b: sp.core.symbol.Symbol | float | int = sp.symbols('b'), 
             i: sp.core.symbol.Symbol | float | int = sp.symbols('i')
            ) -> sp.core.add.Add:
    x_i: sp.core.add.Add = (a + b) / 2 + ((b - a) / 2) * cos(((2*i + 1)/(2*i + 2))*pi)
    return x_i

def get_nodes(a: sp.core.symbol.Symbol | float | int = sp.symbols('a'), 
              b: sp.core.symbol.Symbol | float | int = sp.symbols('b'), 
              n: int = 0) -> list:
    x_i: sp.core.add.Add = get_node(a, b)
    return [x_i.subs(i, i_) for i_ in range(n)]


get_node()

nodes: list = get_nodes(a=inp_data.a, b=inp_data.b, n=inp_data.number_nodes)
    
print(f"Чебышевские узлы для a = {inp_data.a}, b = {inp_data.b}, кол. узлов = {inp_data.number_nodes}")
for node in nodes:
    node

### Глобальный интерполяционный многочлен Фейера, построенный для этой функции на сетке чебышевских узлов

In [ ]:
def get_P(nodes: list | tuple,
          f=None, 
          f_arg: sp.core.symbol.Symbol = sp.symbols('x'), 
          x: sp.core.symbol.Symbol | float | int = sp.symbols('x')
         ) -> sp.core.add.Add:
    
    P: sp.core.add.Add = 0
        
    for x_i in nodes:
        numerator: sp.core.mul.Mul = sp.prod([x - xi_ for xi_ in nodes if xi_ != x_i])
        denominator: sp.core.mul.Mul = sp.prod([x_i - xi_ for xi_ in nodes if xi_ != x_i])

        if not f:
            f_x_i = sp.Function('f')(x_i)
        else:
            f_x_i = f.subs(f_arg, x_i)
            
        P += f_x_i * numerator / denominator

    return P

nodes_symbols: tuple = sp.var(f'x0:{inp_data.number_nodes}')
get_P(nodes_symbols)

print("Для чебышевских узлов и функции f(x): ")
P_x: sp.core.add.Add = get_P(nodes, f_x)
P_x

### График заданной функции `f(x)` и график глобального интерполяционного многочлена Фейера, построенного для этой функции на сетке чебышевских узлов


In [ ]:
plot_1 = sp.plotting.plot(P_x, (x, inp_data.a, inp_data.b), line_color='red', show=False, title='Графики функций f(x) и P(x)') 
plot_2 = sp.plotting.plot(f_x, (x, inp_data.a, inp_data.b), line_color='blue', show=False)
plot_1.append(plot_2[0])
plot_1.show()

### Графики на отдельных изображениях

In [ ]:
sp.plotting.plot(f_x, (x, inp_data.a, inp_data.b), title='График функции f(x)')
sp.plotting.plot(P_x, (x, inp_data.a, inp_data.b), title='График функции P(x)', line_color='red')

### Вся логика вместе, с выводом только графиков

In [ ]:
%%time

try:
    inp_data = InpData.parse_file(Path('input.json'))
except ValidationError as e:
    print("ОШИБКА ВХОДНЫХ ДАННЫХ")
    print(e.json())
    raise e


f_x = x ** 7 - x**6 + 5*x**5 - 3*x**4 + x

nodes: list = get_nodes(a=inp_data.a, b=inp_data.b, n=inp_data.number_nodes)

P_x: sp.core.add.Add = get_P(nodes, f_x)

plot_1 = sp.plotting.plot(P_x, (x, inp_data.a, inp_data.b), line_color='red', show=False, title='Графики функций f(x) и P(x)') 
plot_2 = sp.plotting.plot(f_x, (x, inp_data.a, inp_data.b), line_color='blue', show=False)
plot_1.append(plot_2[0])
plot_1.show()

#### Такой же алгоритм, но немного по другому реализован

In [ ]:
%%time
from sympy.abc import a, b, i, n, x
from sympy import cos, pi
from sympy import var, prod


f = x ** 7 - x**6 + 5*x**5 - 3*x**4 + x
a_ = -10
b_ = 10
n_ = 8


x_i = (a + b) / 2 + ((b - a) / 2) * cos(((2*i + 1)/(2*i + 2))*pi)

nodes: list = [x_i.subs({a: a_, b: b_, i: value}) for value in range(n_)]

P = 0
for i_ in range(n_):
    numerator = prod([x - xi_ for _, xi_ in enumerate(var(f'x0:{n_}')) if _ != i_])
    denominator = prod([var(f'x{i_}') - xi_ for _, xi_ in enumerate(var(f'x0:{n_}')) if _ != i_])

    P += f.subs(x, var(f'x{i_}')) * numerator / denominator
    
d = {}
for i_ in range(n_):
    d[var(f'x{i_}')] = nodes[i_]


P_x = P.subs(d)

plot_1 = sp.plotting.plot(P_x, (x, inp_data.a, inp_data.b), line_color='red', show=False, title='Графики функций f(x) и P(x)') 
plot_2 = sp.plotting.plot(f_x, (x, inp_data.a, inp_data.b), line_color='blue', show=False)
plot_1.append(plot_2[0])
plot_1.show()

#### Такой же алгоритм, но немного по другому реализован

In [ ]:
def get_nodes_not_fast(a_: float | int = None, b_: float | int = None, n_: int =None, x_i=(a + b) / 2 + ((b - a) / 2) * cos(((2*i + 1)/(2*i + 2))*pi)):
    return [x_i.subs({a: inp_data.a, b: inp_data.b, i: i_}) for i_ in range(n_)]

        
get_nodes_not_fast(a_=inp_data.a, b_=inp_data.b, n_=inp_data.number_nodes)

#### Про встроенные функции в `scipy` на эту тему

In [ ]:
# Подробнее тут: https://stackoverflow.com/questions/31543775/how-to-perform-cubic-spline-interpolation-in-python
# или тут: https://translated.turbopages.org/proxy_u/en-ru.ru.e2144e9f-63335301-179385c8-74722d776562/https/www.geeksforgeeks.org/scipy-interpolation/


# Генерация коэффициентов полинома Чебышева в Python
# https://stackoverflow.com/questions/32163523/generating-the-coefficients-of-a-chebyshev-polynomial-in-python

from scipy import interpolate


def foo(x):
    x_points = [0, 1, 2, 3, 4, 5]
    y_points = [12, 14, 22, 39, 58, 77]

    tck = interpolate.splrep(x_points, y_points)
    return interpolate.splev(x, tck)


In [9]:
def get_nodes_not_fast(a_: float | int = None, b_: float | int = None, n_: int =None, x_i=(a + b) / 2 + ((b - a) / 2) * cos(((2*i + 1)/(2*i + 2))*pi)):
    return [x_i.subs({a: inp_data.a, b: inp_data.b, i: i_}) for i_ in range(n_)]

        
get_nodes_not_fast(a_=inp_data.a, b_=inp_data.b, n_=inp_data.number_nodes)

[0,
 -5.0*sqrt(2),
 -5.0*sqrt(3),
 -10.0*sqrt(sqrt(2)/4 + 1/2),
 -10.0*sqrt(sqrt(5)/8 + 5/8),
 -2.5*sqrt(6) - 2.5*sqrt(2),
 -10.0*cos(pi/14),
 -10.0*cos(pi/16)]

#### Про встроенные функции в `scipy` на эту тему

In [11]:
# Подробнее тут: https://stackoverflow.com/questions/31543775/how-to-perform-cubic-spline-interpolation-in-python
# или тут: https://translated.turbopages.org/proxy_u/en-ru.ru.e2144e9f-63335301-179385c8-74722d776562/https/www.geeksforgeeks.org/scipy-interpolation/


# Генерация коэффициентов полинома Чебышева в Python
# https://stackoverflow.com/questions/32163523/generating-the-coefficients-of-a-chebyshev-polynomial-in-python

from scipy import interpolate


def foo(x):
    x_points = [0, 1, 2, 3, 4, 5]
    y_points = [12, 14, 22, 39, 58, 77]

    tck = interpolate.splrep(x_points, y_points)
    return interpolate.splev(x, tck)
